# BentoML service preparation & deployment (local and Docker)

## Setup

Install required dependencies:

In [53]:
!pip3 install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow

[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: pip3 install --upgrade pip


If you are running MacOS use the following pip command:

In [54]:
!pip3 install -r requirements-macos.txt


[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: pip3 install --upgrade pip


In [55]:
!pip3 install "bentoml[io-json]"


[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: pip3 install --upgrade pip


In [56]:
!bentoml -v

bentoml, version 1.0.13


In [57]:
import bentoml
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Reshape
from tensorflow.keras import Model

## Data ingestion & preparation

In [58]:
# import data
df = pd.read_csv('prepared_data.csv')

# remove time column
date_time = pd.to_datetime(df.pop('date'), format='%Y.%m.%d')

In [59]:
df.head()

,temperature,dew_point,humidity,wind,wind_speed,pressure,Cloudy,Cloudy / Windy,Fair,Fair / Windy,...,temperature_seasonal,dew_point_trend,dew_point_seasonal,humidity_trend,humidity_seasonal,wind_speed_trend,wind_speed_seasonal,pressure_trend,pressure_seasonal,revenue
0,3.240741,3.018519,0.984444,225.0,20.555556,979.111111,0,0,0,0,...,-6.478269,3.937704,-2.964012,0.790578,0.176853,12.345752,3.529197,994.209088,-6.961710,3087464.68
1,2.333333,2.175439,0.980351,225.0,13.543860,985.035088,0,0,0,0,...,-6.902768,3.936947,-3.435954,0.790532,0.165998,12.348839,2.839224,994.205078,-6.533516,3087464.68
2,1.923077,1.653846,0.973077,202.5,15.461538,979.807692,0,0,0,0,...,-8.983560,3.936190,-6.358966,0.790486,0.127148,12.351926,3.605715,994.201068,-3.816250,3087464.68
3,2.285714,1.428571,0.928571,157.5,24.040816,973.959184,0,0,0,0,...,-7.757911,3.935433,-4.394456,0.790440,0.161016,12.355013,3.330888,994.197058,-8.478121,2770763.36
4,2.290909,1.618182,0.915818,202.5,23.563636,975.309091,0,0,0,0,...,-7.669534,3.934677,-4.331312,0.790393,0.148989,12.358100,3.117909,994.193048,-7.348745,2802336.20


In [60]:
# helper function to create datasets
def create_dataset(dataset, look_back=31):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        # Take every feature
        a = dataset[i:(i + look_back), :]
        dataX.append(a)
         # Take the last feature (revneue) into window
        dataY.append(dataset[i + look_back, dataset.shape[1]-1])
    return np.array(dataX), np.array(dataY)

In [61]:
# fix random seed for reproducibility
np.random.seed(7)

# load the dataset
dataset = df.values

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# split into train and test sets
train_size = int(len(dataset) * 0.67) 
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

# reshape into windows
look_back = 31
trainX, trainY = create_dataset(train, look_back)  
testX, testY = create_dataset(test, look_back)

# reshape input to be  [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], look_back, len(df.columns)))
testX = np.reshape(testX, (testX.shape[0],look_back, len(df.columns)))

## Define the model

In [62]:
n_forecast = 31
num_features = 1

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.lstm = LSTM(32, return_sequences=False)
        self.dense = Dense(n_forecast*num_features, kernel_initializer=tf.initializers.zeros())
        self.reshape = Reshape([n_forecast, num_features])

    @tf.function(input_signature=[tf.TensorSpec([None, n_forecast, trainX.shape[2]], tf.float32)])
    def call(self, x):
        x = self.lstm(x)
        x = self.dense(x)
        return self.reshape(x)

model = MyModel()

model.compile(loss='mean_squared_error', optimizer='adam')

## Training and Saving the model

Then we initialize some simple tensorflow helper functions and create the training and testing methods

### Training the model

As provided by TensorFlow, we train and test the model.

In [63]:
# TODO: include validation

history = model.fit(
    trainX,
    trainY,
    batch_size=64,
    epochs=20,
    validation_split=0.33,
    # # We pass some validation for
    # # monitoring validation loss and metrics
    # # at the end of each epoch
    # validation_data=(x_val, y_val),
)

#history= model.fit(trainX, trainY,validation_split=0.33, epochs=5, batch_size=32)

Epoch 1/20
13/13 [==============================] - 1s 23ms/step - loss: 0.0175 - val_loss: 0.1875
Epoch 2/20
13/13 [==============================] - 0s 9ms/step - loss: 0.0053 - val_loss: 0.1878
Epoch 3/20
13/13 [==============================] - 0s 8ms/step - loss: 0.0042 - val_loss: 0.1928
Epoch 4/20
13/13 [==============================] - 0s 8ms/step - loss: 0.0042 - val_loss: 0.1885
Epoch 5/20
13/13 [==============================] - 0s 9ms/step - loss: 0.0042 - val_loss: 0.1911
Epoch 6/20
13/13 [==============================] - 0s 8ms/step - loss: 0.0043 - val_loss: 0.1858
Epoch 7/20
13/13 [==============================] - 0s 8ms/step - loss: 0.0044 - val_loss: 0.1882
Epoch 8/20
13/13 [==============================] - 0s 8ms/step - loss: 0.0043 - val_loss: 0.1943
Epoch 9/20
13/13 [==============================] - 0s 8ms/step - loss: 0.0042 - val_loss: 0.1848
Epoch 10/20
13/13 [==============================] - 0s 8ms/step - loss: 0.0042 - val_loss: 0.1873
Epoch 11/20
13/13 

### Saving the model

Make one call to the bentoml library to save this tensorflow model to be used later as part of the prediction service that we will create.

In [64]:
bentoml.tensorflow.save_model(
    "tensorflow_lstm",
    model,
    signatures={"__call__": {"batchable": True, "batch_dim": 0}},
)

INFO:tensorflow:Assets written to: /var/folders/x4/ln2jnbvn4y77s5yjv_4l60p00000gn/T/tmpn7zbb7d_bentoml_model_tensorflow_lstm/assets


INFO:tensorflow:Assets written to: /var/folders/x4/ln2jnbvn4y77s5yjv_4l60p00000gn/T/tmpn7zbb7d_bentoml_model_tensorflow_lstm/assets


Model(tag="tensorflow_lstm:75qxxifhgkwqp4wm", path="/Users/domas/bentoml/models/tensorflow_lstm/75qxxifhgkwqp4wm/")

List all create models in bentoml library.

In [65]:
!bentoml models list

 Tag                        Module              Size        Creation Time       
 tensorflow_lstm:75qxxifh…  bentoml.tensorflow  869.72 KiB  2023-02-08 00:01:39 
 tensorflow_lstm:e6dw4wfh…  bentoml.tensorflow  869.72 KiB  2023-02-07 23:41:18 
 tensorflow_lstm:wgla7rfh…  bentoml.tensorflow  868.21 KiB  2023-02-07 23:38:00 
 tensorflow_lstm:6mqybevh…  bentoml.tensorflow  866.40 KiB  2023-02-07 23:25:31 
 tensorflow_lstm_3:q2vuas…  bentoml.tensorflow  798.40 KiB  2023-02-07 21:13:38 
 tensorflow_lstm_2:gnucaf…  bentoml.tensorflow  887.40 KiB  2023-02-07 20:42:40 
 tensorflow_lstm_2:srazhj…  bentoml.tensorflow  890.29 KiB  2023-02-07 19:19:29 
 tensorflow_mnist:rvdzbwv…  bentoml.tensorflow  890.29 KiB  2023-02-07 19:19:17 
 tensorflow_mnist:rq2o5qv…  bentoml.tensorflow  31.79 MiB   2023-02-07 18:43:28 
 tensorflow_lstm:ahwrjxvh…  bentoml.tensorflow  897.94 KiB  2023-02-07 18:32:26 
 tensorflow_lstm:chv6rhvh…  bentoml.tensorflow  897.94 KiB  2023-02-07 18:18:34 
 tensorflow_lstm:eeucamfg…  

## Create a BentoML Service for serving the model

Note: using `%%writefile` here because `bentoml.Service` instance must be created in a separate `.py` file

Script below creates a single endpoint `predict31`, which will predict from 31 days history into next 31 days.


In [66]:
%%writefile service.py

import bentoml
import numpy as np
import pandas as pd
from bentoml.io import Image, NumpyNdarray, JSON
from sklearn.preprocessing import MinMaxScaler
from pydantic import BaseModel

runner = bentoml.tensorflow.get("tensorflow_lstm:latest").to_runner()

svc = bentoml.Service(
    name="tensorflow_lstm_demo",
    runners=[runner],
)

@svc.api(input=JSON(), output=NumpyNdarray(dtype="float32"))
async def predict31(input_data: JSON) -> "np.ndarray":

    # Ingest data
    dataset = pd.DataFrame(input_data)
    dataset = dataset.T.values

    print(dataset.shape)

    # Normalize
    sc = MinMaxScaler(feature_range=(0, 1))
    scaled = sc.fit_transform(dataset)

    # Reshape
    test = scaled[:, :dataset.shape[1]-1]
    testX = np.array(test)
    testX = np.reshape(testX, (1, 31, dataset.shape[1]-1))

    testPredict = await runner.async_run(testX)

    testPredict_extended = np.zeros((31, dataset.shape[1]))
    testPredict_extended[:,dataset.shape[1] - 1] = testPredict[:, :, 0]
    testPredict = sc.inverse_transform(testPredict_extended)[:,dataset.shape[1] - 1]

    return testPredict 

Overwriting service.py


Start a dev model server with the comand below (for local testing)

In [47]:
# !bentoml serve service.py:svc --reload

2023-02-07T23:45:22+0200 [INFO] [cli] Prometheus metrics for HTTP BentoServer from "service.py:svc" can be accessed at http://localhost:3000/metrics.
2023-02-07T23:45:23+0200 [INFO] [cli] Starting development HTTP BentoServer from "service.py:svc" listening on http://0.0.0.0:3000 (Press CTRL+C to quit)
2023-02-07 23:45:23 circus[18363] [INFO] Loading the plugin...
2023-02-07 23:45:23 circus[18363] [INFO] Endpoint: 'tcp://127.0.0.1:50050'
2023-02-07 23:45:23 circus[18363] [INFO] Pub/sub: 'tcp://127.0.0.1:50052'
2023-02-07T23:45:23+0200 [INFO] [observer] Watching directories: ['/Users/domas/Desktop/code/barbora/bentoml_2/working', '/Users/domas/bentoml/models']
2023-02-07T23:47:14+0200 [INFO] [dev_api_server:tensorflow_lstm_demo] 127.0.0.1:50115 (scheme=http,method=-H,path=/predict31,type=application/x-www-form-urlencoded,length=34997) (status=405,type=text/plain; charset=utf-8,length=18) 7.113ms (trace=8e0eb9ab2b53b88cbdbc8bc844ac4e3f,span=9b56cfe830222052,sampled=0)
2023-02-07T23:47:49

### Test local endpoint

If using file from data notebook

`curl -X POST -H "Content-Type: application/json" -d @test.json http://0.0.0.0:3000/predict31`

or alternatively, python request:

In [ ]:
### test from other notebook, since server is running here

import requests

url = 'http://0.0.0.0:3000/predict31'
headers = {"content-type": "application/json"}
r = requests.post(url, data=open('file-3.json', 'rb'), headers=headers).text

print(r)

## Build a Bento for distribution and deployment

Note: if running, stop the server above to proceed

A `bentofile` is already created in this directory  with all the necessary packages for building a Bento for the service:

```yaml
service: "service:svc"
description: "file: ./README.md"
labels:
  owner: Domas
  stage: demo
include:
  - "*.py"
exclude:
  - "locustfile.py"
python:
  lock_packages: false
  packages:
    - tensorflow
    - Pillow
    - pandas
    - scikit-learn
    - pydantic

```

Run `bentoml build` from current directory to build a Bento with the latest version of the `tensorflow_lstm_demo` model. This may take a while when running for the first time for BentoML to resolve all dependency versions:

In [49]:
!bentoml build

Building BentoML service "tensorflow_lstm_demo:7dedq6vhggtwr4wm" from build context "/Users/domas/Desktop/code/barbora/bentoml_2/working".
Packing model "tensorflow_lstm:e6dw4wfhgcwqp4wm"

██████╗░███████╗███╗░░██╗████████╗░█████╗░███╗░░░███╗██╗░░░░░
██╔══██╗██╔════╝████╗░██║╚══██╔══╝██╔══██╗████╗░████║██║░░░░░
██████╦╝█████╗░░██╔██╗██║░░░██║░░░██║░░██║██╔████╔██║██║░░░░░
██╔══██╗██╔══╝░░██║╚████║░░░██║░░░██║░░██║██║╚██╔╝██║██║░░░░░
██████╦╝███████╗██║░╚███║░░░██║░░░╚█████╔╝██║░╚═╝░██║███████╗
╚═════╝░╚══════╝╚═╝░░╚══╝░░░╚═╝░░░░╚════╝░╚═╝░░░░░╚═╝╚══════╝

Successfully built Bento(tag="tensorflow_lstm_demo:7dedq6vhggtwr4wm").


### Dockerize

Put `Bento` into `Docker` container.
Make sure the docker service itself is running before start.
This might take some time.

In [50]:
!bentoml containerize tensorflow_lstm_demo:latest

Building OCI-compliant image for tensorflow_lstm_demo:7dedq6vhggtwr4wm with docker

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (0/1)                                                         
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 35B                                        0.0s
[+] Building 0.2s (2/2)                                                         
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 1.66kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
[+] Building 0.4s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 

### Run service in Docker container

After container is prepared, the very last line shows a default command to run docker container. Run it to start the service.

In [51]:
# run using described command above
# similar to this -> 
# !docker run -it --rm -p 3000:3000 tensorflow_lstm_demo:djls6yvhbspzn4wm serve --production

!docker run -it --rm -p 3000:3000 tensorflow_lstm_demo:7dedq6vhggtwr4wm serve --production

/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: cannot open shared object file: No such file or directory']
  warnings.warn(f"file s

### Test Docker endpoint

If using file from data notebook

`curl -X POST -H "Content-Type: application/json" -d @test.json http://0.0.0.0:3000/predict31`

or alternatively, python request:

In [ ]:
### test from other notebook, since server is running here

import requests

url = 'http://0.0.0.0:3000/predict31'
headers = {"content-type": "application/json"}
r = requests.post(url, data=open('file-3.json', 'rb'), headers=headers).text

print(r)